In [3]:
import numpy as np
from numpy import absolute, mean, std
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline

import random

random.seed(10)

# https://machinelearningmastery.com/rfe-feature-selection-in-python/

In [4]:
#work_p = "C:/Users/claire.smid/Documents/Main_STUDY/Model-based_learning_mainstudy/"

work_p = "C:/Users/claire.smid/Documents/DCP/Writing/Thesis/Chapter2/"

# Including all decision making, EFs and mental health:
#dataset = pd.read_csv(work_p+"All_DMs_EFs_MHs_Nov21.csv", sep=',') 

# Including all decision making and EFs
#dataset = pd.read_csv(work_p+"All_DMs_EFs_Nov21.csv", sep=',') 

# Including EFs and MB only
#dataset = pd.read_csv(work_p+"MB_EFs_Only_Nov21.csv", sep=',') 
#dataset = pd.read_csv(work_p+"MB_EF_Detail_Imputed_Dec21.csv", sep=',') 
#dataset = pd.read_csv(work_p+"MB_EF_Detail_Imputed_REDUCED_Dec21.csv", sep=',') 
#dataset = pd.read_csv(work_p+"MBMF_EF_Detail_Imp_6Jun2022.csv", sep=',') 
dataset = pd.read_csv(work_p+"MBMF_EF_Detail_Imp_9Jun2022.csv", sep=',') 

pd.options.display.max_columns = 100

dataset

,ID,Session,School,Gender,Age_Frac_Imp,SES_inv_z,Avg_Pts,Avg_Pts_lo,Avg_Pts_hi,w_P6,w_diff,it_P6,lr_P6,eg_P6,st_P6,repst_P6,it,lr,eg,st,repst,T_Vocab,T_Matrix,Corsi_WM_Span,CogFlex_t,SSRT,FlankerSwitch_t,FlankerInhib_t,Stroop_t,AY_RCS,BX_RCS,Nback_1_RCS,Nback_2_RCS,CF_Mix_RCS_Diff,SSRT_SSD_z
0,1,0,Trevor Roberts School,M,12.444444,0.226732,0.013173,-0.002222,0.029551,0.425788,0.121567,0.619787,0.437227,0.419932,-0.177439,-0.709067,0.625122,0.442486,0.420558,-0.177359,-0.704026,62,66,7,0.524022,-0.798678,0.035289,-0.303618,-0.818045,1.785449,0.927080,0.241960,0.114083,-0.699165,0.316186
1,4,0,Trevor Roberts School,F,11.838889,-0.587075,0.024752,0.004831,0.041414,0.528114,-0.142475,0.804932,0.698279,0.486952,-0.435025,-0.862134,0.812849,0.685928,0.486859,-0.426516,-0.851632,55,50,5,1.152471,0.028908,0.242714,0.563637,-0.013837,0.985358,1.401536,0.272975,0.123974,-0.018518,-0.640517
2,5,0,Trevor Roberts School,M,7.611111,1.040539,0.043889,0.037037,0.052972,0.770192,0.132597,0.880334,0.755154,0.530449,-0.589862,-0.083494,0.846292,0.747178,0.556262,-0.601976,-0.080824,66,55,4,-1.093324,-0.380691,0.525759,-0.140001,-0.205424,0.990982,0.373066,0.117747,0.174845,-0.171262,0.880708
3,8,0,Trevor Roberts School,M,6.747222,0.226732,0.013740,-0.022676,0.054293,0.582383,0.028154,0.651551,0.764537,0.577141,0.588800,0.396509,0.652178,0.783163,0.582775,0.589636,0.395179,64,61,4,0.458539,-1.489266,-0.041334,-0.248403,1.083356,0.562811,0.437377,0.102242,0.228547,-0.138648,-0.071924
4,15,0,Trevor Roberts School,M,10.600000,0.226732,-0.014891,0.016414,-0.040881,0.448529,-0.036160,0.553742,0.489282,0.530107,-0.303803,-1.629661,0.553467,0.487266,0.528799,-0.305209,-1.630474,55,62,6,-0.133400,0.016327,-0.679340,-0.246747,0.085989,0.461337,0.558434,0.216780,0.260057,0.025455,-1.590435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,193,0,Broadfields,M,10.486111,-2.214688,0.098073,0.071605,0.120545,0.422117,0.117357,0.433004,0.519166,0.491737,0.173996,-0.269121,0.436899,0.517092,0.471755,0.175653,-0.262585,41,56,6,0.329597,0.200856,-0.572747,0.212403,0.187235,1.589251,2.482398,0.174274,0.156276,0.034773,1.374665
65,195,0,Merton Court,M,10.372222,-0.587075,-0.017052,-0.065302,0.045455,0.610913,-0.218845,0.445750,0.349609,0.394229,-0.074491,-0.851411,0.453946,0.367773,0.398445,-0.070527,-0.840291,58,64,6,0.680336,-0.707812,-0.136637,0.689489,-0.980713,0.913796,1.094341,0.217520,0.150937,-0.266687,-1.236251
66,196,0,Orley Farm,M,8.608333,0.498001,0.057756,0.049645,0.064815,0.718295,0.107204,1.381509,0.868366,0.658000,0.382585,-0.281852,1.387507,0.884099,0.678044,0.386831,-0.277990,64,52,5,-0.330512,0.909617,0.208257,-0.148340,-0.622866,1.036768,0.302673,0.209424,0.273568,-0.564426,0.139772
67,202,0,Newland House,F,11.236111,1.040539,0.044004,0.045351,0.042735,0.893768,0.043038,1.296684,0.839617,0.514458,0.758670,-0.065990,1.165809,0.808938,0.521772,0.873589,-0.060433,63,54,7,0.458539,-0.562425,-0.814565,0.551108,-0.565334,0.818956,0.818888,0.140716,0.135013,0.012360,-1.571437


In [5]:
# one hot encoding
dataset.Gender.replace(to_replace=['M','F'],value=[1,0],inplace=True)

In [6]:
print(len(dataset.columns))
print(list(dataset.columns))
cols = dataset.columns

35
['ID', 'Session', 'School', 'Gender', 'Age_Frac_Imp', 'SES_inv_z', 'Avg_Pts', 'Avg_Pts_lo', 'Avg_Pts_hi', 'w_P6', 'w_diff', 'it_P6', 'lr_P6', 'eg_P6', 'st_P6', 'repst_P6', 'it', 'lr', 'eg', 'st', 'repst', 'T_Vocab', 'T_Matrix', 'Corsi_WM_Span', 'CogFlex_t', 'SSRT', 'FlankerSwitch_t', 'FlankerInhib_t', 'Stroop_t', 'AY_RCS', 'BX_RCS', 'Nback_1_RCS', 'Nback_2_RCS', 'CF_Mix_RCS_Diff', 'SSRT_SSD_z']


In [5]:
# # 6 jun dataset
# X = dataset.iloc[:, np.r_[14:34]]
# X_cols = X
# y = dataset.iloc[:,8]

# 7 jun dataset
X = dataset.iloc[:, np.r_[21:42]]
X_cols = X
y = dataset.iloc[:,7]

In [6]:
#X = X.reset_index(drop=True)
X

,T_Vocab,T_Matrix,Corsi_WM_Span,AXCPT_t,CogFlex_SwitchRT_t,SSRT,FlankerSwitch_t,FlankerInhib_t,Stroop_t,axcpt_PBI_t0,AXCPT_CorrRT_Shift_Index,AY_RCS,BX_RCS,Flank_Switch_RCS_Diff,Flank_InCon_RCS_Diff,Nback_1_RCS,Nback_2_RCS,CF_Switch_RCS_Diff,CF_Mix_RCS_Diff,SSRT_SSD_z,Corsi_CorrRT_log
0,62,66,7,-0.696260,176.281070,-0.798678,0.035289,-0.303618,-0.818045,-0.696260,0.260224,1.785449,0.927080,-0.308600,-0.498360,0.241960,0.114083,-0.405414,-0.699165,0.316186,-7.521572
1,55,50,5,0.431760,93.596497,0.028908,0.242714,0.563637,-0.013837,0.431760,0.371609,0.985358,1.401536,-0.264254,-0.086407,0.272975,0.123974,-0.720982,-0.018518,-0.640517,-7.302517
2,66,55,4,-0.433764,123.828610,-0.380691,0.525759,-0.140001,-0.205424,-0.433764,0.056031,0.990982,0.373066,0.011101,-0.140620,0.117747,0.174845,0.054496,-0.171262,1.903905,-7.973345
3,64,61,4,-0.614938,800.391720,-1.489266,-0.041334,-0.248403,1.083356,-0.063924,0.029162,0.562811,0.437377,-0.314666,-0.261631,0.102242,0.228547,0.040507,0.057369,-0.071924,-8.065688
4,55,62,6,-0.597904,195.210570,0.016327,-0.679340,-0.246747,0.085989,-0.597904,0.121451,0.461337,0.558434,-0.714397,-0.277322,0.216780,0.260057,-0.072714,0.025455,-1.590435,-7.684010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,41,56,6,-0.059368,182.111080,0.200856,-0.572747,0.212403,0.187235,-0.059368,0.219358,1.589251,2.482398,-0.671223,-0.117612,0.174274,0.156276,-0.509668,0.034773,1.374665,-7.365308
65,58,64,6,-0.064548,349.235350,-0.707812,-0.136637,0.689489,-0.980713,-0.064548,0.089907,0.913796,1.094341,-0.225866,0.051139,0.217520,0.150937,-0.149499,-0.266687,-1.236251,-7.367070
66,64,52,5,-0.435616,437.055540,0.909617,0.208257,-0.148340,-0.622866,0.600552,0.008661,1.036768,0.302673,-0.125693,-0.081658,0.209424,0.273568,0.040507,-0.571175,0.139772,-7.444318
67,63,54,7,-0.438972,82.833313,-0.562425,-0.814565,0.551108,-0.565334,-0.438972,-0.026887,0.818956,0.818888,-0.770009,0.112121,0.140716,0.135013,-0.511400,0.012360,-1.571437,-7.552011


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# LOOCV with all features

In [9]:
# find number of samples
n = X.shape[0]
# create loocv procedure
cv = LeaveOneOut()
# create model
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
# evaluate model
scores = cross_val_score(regressor, X, y, scoring='neg_mean_squared_error', cv=n)
# force positive
#scores = absolute(scores)

# report performance
print('MSE: %.3f (%.3f)' % (mean(scores), std(scores)))


MSE: -0.072 (0.086)


# First RFE

In [10]:
# explore the algorithm wrapped by RFE
# automatically select the number of features for RFE

# create pipeline
rfe = RFECV(estimator=DecisionTreeRegressor(random_state = 0),min_features_to_select=1)
model = DecisionTreeRegressor(random_state = 0)
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# find number of samples
n = X.shape[0]
# create loocv procedure
cv = LeaveOneOut()
# evaluate model
scores = cross_val_score(pipeline, X, y, scoring='neg_mean_squared_error', cv=n)
# report performance
print('MSE: %.3f (%.3f)' % (mean(scores), std(scores)))

MSE: -0.068 (0.101)


In [11]:
# find out which predictors
rfe.fit(X,y)
data_top = X_cols.columns
# summarise all features
for i in range(X.shape[1]):
    print('Selected %s,\t Rank: %.1f, \tName: %s' % (rfe.support_[i], rfe.ranking_[i],data_top[i]))

Selected False,	 Rank: 3.0, 	Name: T_Vocab
Selected False,	 Rank: 6.0, 	Name: T_Matrix
Selected False,	 Rank: 17.0, 	Name: Corsi_WM_Span
Selected False,	 Rank: 14.0, 	Name: AXCPT_t
Selected False,	 Rank: 16.0, 	Name: CogFlex_SwitchRT_t
Selected False,	 Rank: 12.0, 	Name: SSRT
Selected False,	 Rank: 2.0, 	Name: FlankerSwitch_t
Selected False,	 Rank: 11.0, 	Name: FlankerInhib_t
Selected True,	 Rank: 1.0, 	Name: Stroop_t
Selected False,	 Rank: 9.0, 	Name: axcpt_PBI_t0
Selected False,	 Rank: 20.0, 	Name: AXCPT_CorrRT_Shift_Index
Selected False,	 Rank: 13.0, 	Name: AY_RCS
Selected False,	 Rank: 18.0, 	Name: BX_RCS
Selected False,	 Rank: 19.0, 	Name: Flank_Switch_RCS_Diff
Selected True,	 Rank: 1.0, 	Name: Flank_InCon_RCS_Diff
Selected False,	 Rank: 15.0, 	Name: Nback_1_RCS
Selected False,	 Rank: 4.0, 	Name: Nback_2_RCS
Selected False,	 Rank: 8.0, 	Name: CF_Switch_RCS_Diff
Selected False,	 Rank: 7.0, 	Name: CF_Mix_RCS_Diff
Selected False,	 Rank: 10.0, 	Name: SSRT_SSD_z
Selected False,	 Rank: 

# Permutation testing

In [12]:
from sklearn.model_selection import RepeatedKFold
import eli5
from eli5.sklearn import PermutationImportance
from eli5 import show_prediction, show_weights

Xfeature_names = X_cols.columns

# create pipeline
regressor = DecisionTreeRegressor(random_state = 0)
cv = RepeatedKFold(n_splits=6, n_repeats=10, random_state=1)
perm = PermutationImportance(regressor, cv = cv)
perm.fit(X,y)

show_weights(perm, feature_names = X_cols.columns.tolist())

C:\Users\claire.smid\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classifier=False as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Weight,Feature
0.1059 ± 1.3713,Flank_Switch_RCS_Diff
0.0381 ± 0.4652,AY_RCS
0.0361 ± 2.2455,Stroop_t
0.0290 ± 0.7751,AXCPT_t
0.0094 ± 0.4862,SSRT
0.0067 ± 0.2984,Corsi_WM_Span
0.0054 ± 0.3649,Nback_1_RCS
0.0050 ± 0.4702,axcpt_PBI_t0
0.0049 ± 0.8742,Corsi_CorrRT_log
-0.0036 ± 0.2405,CF_Mix_RCS_Diff


# Hyperparameter tuning

In [13]:
regressor.get_params().keys()

dict_keys(['ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])

In [14]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

X = dataset[["Flank_Switch_RCS_Diff","AY_RCS","Stroop_t","AXCPT_t"]]
X_cols = X

X = scaler.fit_transform(X)

# find number of samples
n = X.shape[0]
# create loocv procedure
cv = LeaveOneOut()
# create model
regressor = DecisionTreeRegressor(random_state = 0)
# parameters
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 5)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 15, num = 10)]
max_depth.append(None)
min_samples_split = [int(x) for x in np.linspace(1, 10, num = 6)]
min_samples_leaf = [int(x) for x in np.linspace(1, 6, num = 4)]
# define search space
space = dict()
#space["n_estimators"] = n_estimators
space["max_features"] = ['auto', 'sqrt']
space['max_depth'] = max_depth
space["min_samples_split"] = min_samples_split
space["min_samples_leaf"] = min_samples_leaf
space['ccp_alpha'] =  [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
space['splitter'] = ["best","random"]
#space["bootstrap"] = [True, False] 
space["max_leaf_nodes"] = [None, 10, 20, 30, 40, 50, 60, 70, 80, 90]
# define search
search = GridSearchCV(regressor,space,scoring='neg_mean_squared_error',cv=cv, n_jobs = -1)
# execute search
result = search.fit(X,y)

# summarize result
print('Best MSE Score: %s' % result.best_score_)
print('Best Hyperparameters %s' % result.best_params_ )

C:\Users\claire.smid\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [        nan         nan -0.03196365 ... -0.03739814 -0.03739814
 -0.03739814]
  warnings.warn(


Best MSE Score: -0.028759666728432047
Best Hyperparameters {'ccp_alpha': 1e-05, 'max_depth': 4, 'max_features': 'auto', 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_samples_split': 4, 'splitter': 'random'}


In [ ]:
# explore the algorithm wrapped by RFE
# automatically select the number of features for RFE
from sklearn.model_selection import RepeatedKFold

# create pipeline
from sklearn.ensemble import RandomForestRegressor
regressor = DecisionTreeRegressor(random_state = 0, n_estimators = , max_depth = , max_features = ,
                                 )

# evaluate model
cv = RepeatedKFold(n_splits=6, n_repeats=3, random_state=1)
n_scores = cross_val_score(regressor, X, y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
r2_scores = cross_val_score(regressor, X, y, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
explained_var = cross_val_score(regressor, X, y, scoring='explained_variance', cv=cv, n_jobs=-1, error_score='raise')

# report performance
print('MSE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print('r2: %.3f (%.3f)' % (mean(r2_scores), std(r2_scores)))
print('exp var: %.3f' % mean(explained_var))


## new results:
### rbf kernel (two predictors: flanker and stroop) (7 jun data)
- MSE: 
- r2: 
- exp var: 

### 4 predictors (flanker, axcpt (2) and stroop (7 jun data)
- MSE: -0.034
- r2: -0.144
- exp var: -0.038


In [ ]:
Xfeature_names = X_cols.columns

# create pipeline
regressor = SVR(kernel = 'rbf')

cv = RepeatedKFold(n_splits=6, n_repeats=100, random_state=1)
perm = PermutationImportance(regressor, cv = cv)
perm.fit(X,y)

show_weights(perm, feature_names = X_cols.columns.tolist())
#show_prediction(perm, X.iloc[4], feature_names = X.columns.tolist())